<a href="https://colab.research.google.com/github/IshuDhana/lab-chains-in-langchain/blob/main/lab_chains_in_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
from google.colab import userdata
import os
# Retrieve the API key from Colab secrets
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# Set it as an environment variable, which LangChain often uses by default
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
print("OpenAI API key loaded and set as environment variable.")

OpenAI API key loaded and set as environment variable.


In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
df = pd.read_csv('/content/Data.csv')

In [ ]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
#!pip install langchain_community

In [ ]:
!pip install langchain langchain-openai python-dotenv

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.4 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [ ]:
#from langchain_openai import ChatOpenAI
#from langchain.prompts import ChatPromptTemplate
#from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(temperature=1)

In [ ]:
prompt = ChatPromptTemplate.from_template('List the positive reviews about the {product}')

In [ ]:

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = 'Queen Size Sheet Set' #Select a product type to be describe
chain.run(product)

'1. "I absolutely love my Queen Size Sheet Set! The sheets are soft, cozy, and fit perfectly on my mattress. Definitely recommend."\n\n2. "The quality of these sheets is top-notch. They are durable, comfortable, and have held up well after multiple washes. Will be purchasing more for my other bedrooms."\n\n3. "I am beyond impressed with the Queen Size Sheet Set. The color is vibrant, the material is silky smooth, and they stay in place throughout the night. So happy with my purchase."\n\n4. "These sheets are everything I was looking for and more. They are breathable, lightweight, and have a perfect fit. I look forward to climbing into bed every night thanks to these sheets."\n\n5. "I cannot say enough good things about the Queen Size Sheet Set. They are luxurious, soft, and have a beautiful sheen to them. I feel like I\'m sleeping in a 5-star hotel every night."\n\n6. "The Queen Size Sheet Set exceeded my expectations. The deep pockets on the fitted sheet are a game-changer, and the ov

## SimpleSequentialChain

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes the {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company: {company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Regal Linens Co.
Regal Linens Co. offers luxurious and high-quality bedding and linens, providing comfort and elegance for your home decor needs.

> Finished chain.


'Regal Linens Co. offers luxurious and high-quality bedding and linens, providing comfort and elegance for your home decor needs.'

**Repeat the above twice for different products**

In [ ]:
product = 'Luxury Air Mattress' #Select a product type to be describe
chain.run(product)

llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the worst name to describe a company that makes the {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company: {company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
overall_simple_chain.run(product)



> Entering new MultiPromptChain chain...
None: {'input': 'Luxury Air Mattress'}
> Finished chain.


> Entering new SimpleSequentialChain chain...
"Ultimate Deflation"
Ultimate Deflation is a financial consulting firm specializing in strategies to navigate and thrive in periods of economic deflation.

> Finished chain.


'Ultimate Deflation is a financial consulting firm specializing in strategies to navigate and thrive in periods of economic deflation.'

## SequentialChain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

Review = "Les ordinateurs portables GamersTech impressionne par ses performances  exceptionnelles et son design elegant."

first_prompt = ChatPromptTemplate.from_template(
  "Translate the following review to english: \n\n {Review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review" #Give a name to your output
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summmarize the following review in 1 sentence: \n\n {English_Review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary" #give a name to this output
                    )


In [ ]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review: \n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="Language"
                      )


In [ ]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "What is the language {Review}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="Language"
                     )


In [ ]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "Language"],
    verbose=True
)

In [ ]:
review = df.Review[5]
overall_chain(Review)

/tmp/ipython-input-3497060622.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(Review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Les ordinateurs portables GamersTech impressionne par ses performances  exceptionnelles et son design elegant.',
 'English_Review': 'The GamersTech laptops impress with their exceptional performance and elegant design.',
 'summary': 'GamersTech laptops stand out for their impressive performance and sleek design.',
 'Language': 'French'}

**Repeat the above twice for different products or reviews**

In [ ]:
llm = ChatOpenAI(temperature=0.9)

Review = "Les ordinateurs portables GamersTech combinent des performances de haut niveau avec un design élégant et contemporain."

first_prompt = ChatPromptTemplate.from_template(
  "Translate the following review to english: \n\n {Review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review" #Give a name to your output
                    )

second_prompt = ChatPromptTemplate.from_template(
    "Can you summmarize the following review in 1 sentence: \n\n {English_Review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary" #give a name to this output
                    )

# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review: \n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="Language"
                      )
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "What is the language {Review}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="Language"
                     )

# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "Language"],
    verbose=True
)

review = df.Review[5]
overall_chain(Review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Les ordinateurs portables GamersTech combinent des performances de haut niveau avec un design élégant et contemporain.',
 'English_Review': 'GamersTech laptops combine high-performance with a sleek and contemporary design.',
 'summary': 'GamersTech laptops offer both impressive performance and a stylish modern aesthetic.',
 'Language': 'French'}

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipython-input-2333002322.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [ ]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [ ]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for the compliment! The answer to the question "what is 2 + 2" is 4.'

In [ ]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures the proper functioning of the organism as a whole.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**